# Исследование текущего уровня пользовательской лояльности среди клиентов из России

## Описание проекта

**Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.**

In [1]:
import os
import pandas as pd
import numpy as np


from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
SELECT u.user_id,
       u.lt_day,
       CASE WHEN u.lt_day <= 365 THEN 'новый клиент' ELSE 'старый клиент' END AS is_new,
       u.age,
       CASE WHEN u.gender_segment = 1 THEN 'женщина' 
            WHEN u.gender_segment = 0 THEN 'мужчина' 
            ELSE 'не определен'
       END as gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       SUBSTR(ags.title, 4) AS age_segment,
       SUBSTR(trs.title, 4) AS traffic_segment,
       SUBSTR(lts.title, 4) AS lifetime_segment,
       u.nps_score,
       CASE
           WHEN u.nps_score <= 6 THEN 'detractors'
           WHEN u.nps_score >= 9 THEN 'promoters'
           ELSE 'passives'
       END AS nps_group    
FROM USER AS u
JOIN LOCATION AS l ON l.LOCATION_ID = u.LOCATION_ID
JOIN AGE_SEGMENT AS ags ON ags.AGE_GR_ID = u.AGE_GR_ID
JOIN TRAFFIC_SEGMENT AS trs ON trs.TR_GR_ID = u.TR_GR_ID
JOIN LIFETIME_SEGMENT AS lts ON lts.LT_GR_ID = u.LT_GR_ID
"""

In [4]:
df = pd.read_sql(query, engine)
display(df['gender_segment'].unique())
display(df.head(10))

array(['женщина', 'мужчина', 'не определен'], dtype=object)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,старый клиент,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,promoters
1,A001WF,2344,старый клиент,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,promoters
2,A003Q7,467,старый клиент,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,promoters
3,A004TB,4190,старый клиент,44.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,promoters
4,A004XT,1163,старый клиент,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,promoters
5,A005O0,5501,старый клиент,42.0,женщина,ANDROID,SMARTPHONE,Россия,Омск,35-44,5-10,36+,6,detractors
6,A0061R,1236,старый клиент,45.0,мужчина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,10-15,36+,10,promoters
7,A009KS,313,новый клиент,35.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,35-44,45-50,7-12,10,promoters
8,A00AES,3238,старый клиент,36.0,женщина,ANDROID,SMARTPHONE,Россия,СанктПетербург,35-44,1-5,36+,10,promoters
9,A00F70,4479,старый клиент,54.0,женщина,ANDROID,SMARTPHONE,Россия,Волгоград,45-54,15-20,36+,9,promoters


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

## Ссылки на дашборд и презентацию

Ссылка на дашборд - https://public.tableau.com/app/profile/alexey.kosheev/viz/2_17083573675090/sheet0?publish=yes

Ссылка на презентацию - https://docs.google.com/presentation/d/1jfEijHcdJg-uCW3JhGRuw_FVXydASFyVbk5y50msO9g/edit?usp=sharing